In [85]:
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

In [86]:
commodity_ids = {
  'Heating': 6,
  'Cooling': 7
}

inv_commodity_ids = {v: k for k, v in commodity_ids.items()}

# Read in the files
assets = pd.read_csv('../data/asset.csv', sep='|')
data = pd.read_csv('../output/output_TH.tsv', sep='\t')

# Initialize variables
start_ts = datetime(2024, 1, 1)
time_increment = timedelta(hours=1)

# Separate columns for Heating and Cooling
heating_columns = [col for col in data.columns if 'Heating(Wh)' in col]
cooling_columns = [col for col in data.columns if 'Cooling(Wh)' in col]

# Create separate dataframes for Heating and Cooling
heating_data = data[heating_columns].copy()
cooling_data = data[cooling_columns].copy()

# Initialize lists to store results
heating_rows = []
cooling_rows = []

In [87]:
# Process each row in the TSV data
for index, row in data.iterrows():
    current_ts = start_ts + index * time_increment
    
    for col in heating_columns:
        building_id = col.split('(')[0]
        heating_rows.append({
            'asset_id': building_id,
            'commodity_id': commodity_ids['Heating'],
            'ts': current_ts,
            'is_prediction': 't',
            'value': row[col] / 1000  # Convert Wh to kWh
        })
    
    for col in cooling_columns:
        building_id = col.split('(')[0]
        cooling_rows.append({
            'asset_id': building_id,
            'commodity_id': commodity_ids['Cooling'],
            'ts': current_ts,
            'is_prediction': 't',
            'value': row[col] / 1000  # Convert Wh to kWh
        })


In [88]:
# Convert lists to dataframes
heating_df = pd.DataFrame(heating_rows)
cooling_df = pd.DataFrame(cooling_rows)

# Combine the dataframes
combined_df = pd.concat([heating_df, cooling_df], ignore_index=True)
combinded_df_month = combined_df.copy()

In [89]:
# Truncate dates to month and sum values
combinded_df_month['month'] = combinded_df_month['ts'].dt.to_period('M')
monthly_sum_df = combinded_df_month.groupby(['asset_id', 'commodity_id', 'month']).agg({'value': 'sum'}).reset_index()

# Plot each asset and save the plots
for asset_id, asset_group in monthly_sum_df.groupby('asset_id'):
    asset_name = assets.loc[assets['id'] == int(asset_id), 'name'].values[0]
    asset_name_fixed = asset_name.replace(' ', '_')
    plt.figure()
    for commodity_id, commodity_group in asset_group.groupby('commodity_id'):
        color = 'red' if commodity_id == 6 else 'blue'
        label = 'Heating' if commodity_id == 6 else 'Cooling'
        plt.plot(commodity_group['month'].astype(str), commodity_group['value'], marker='o', color=color, label=f'{label} Sum')
        
        # Calculate and plot the average value
        average_value = commodity_group['value'].mean()
        plt.axhline(y=average_value, color=color, linestyle='--', label=f'{label} Average')
    
    plt.title(f'{asset_name}')
    plt.xlabel('Month')
    plt.ylabel('Value (kWh)')
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.savefig(f'../output/plots/{asset_name_fixed}.png')
    plt.close()


In [90]:
# Save the combined dataframe as a CSV file
combined_df.to_csv('../output/measurement.csv', sep='|', index=False)

# Print or display the combined dataframe for verification
print(combined_df.head())

  asset_id  commodity_id         ts is_prediction     value
0       82             6 2024-01-01             t   428.123
1       81             6 2024-01-01             t   180.450
2       79             6 2024-01-01             t    66.635
3       78             6 2024-01-01             t   477.515
4       77             6 2024-01-01             t  1148.475
